In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

train.head(3)
train.tail(3)

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
4749,4749,19.7,987.7,0.9,0.6,0.0,1006.9,987.6,21.4,1007.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4750,4750,19.4,987.7,0.9,0.8,0.0,1006.9,987.8,21.3,1007.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
4751,4751,19.1,987.6,1.0,0.3,0.0,1006.8,987.8,21.2,1007.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0


In [ ]:
"""
train['date'] = pd.PeriodIndex(start='2020-01-01', end=None, periods=len(train), freq='10Min')
test['date'] = pd.PeriodIndex(start='2020-02-03', end=None, periods=len(test), freq='10Min')
train.head(15)

train['minute'] = (train['id']%6)*10
train.head(15)
"""

In [3]:
train['hour'] = (train['id']//6) - (train['id']//144)*24
test['hour'] = (test['id']//6) - (test['id']//144)*24

In [4]:
train = pd.get_dummies(train, columns=['hour'], prefix='H_')
test = pd.get_dummies(test, columns=['hour'], prefix='H_')
train.head(15)

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,H__14,H__15,H__16,H__17,H__18,H__19,H__20,H__21,H__22,H__23
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,...,0,0,0,0,0,0,0,0,0,0
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,...,0,0,0,0,0,0,0,0,0,0
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,...,0,0,0,0,0,0,0,0,0,0
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,...,0,0,0,0,0,0,0,0,0,0
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,...,0,0,0,0,0,0,0,0,0,0
5,5,9.1,988.9,1.3,0.7,0.0,1009.1,989.6,12.0,1010.0,...,0,0,0,0,0,0,0,0,0,0
6,6,9.1,988.8,2.4,1.9,0.0,1009.1,989.6,12.0,1010.1,...,0,0,0,0,0,0,0,0,0,0
7,7,8.9,988.9,1.0,1.2,0.0,1009.2,989.7,11.9,1010.1,...,0,0,0,0,0,0,0,0,0,0
8,8,9.0,988.9,1.5,0.1,0.0,1009.2,989.6,11.9,1010.0,...,0,0,0,0,0,0,0,0,0,0
9,9,8.8,988.9,1.4,0.4,0.0,1009.4,989.6,11.8,1010.1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def same_min_max(df):
    return df.drop(df.columns[df.max() == df.min()], axis=1, inplace=True)

In [6]:
same_min_max(train)
same_min_max(test)

In [7]:
# 평균기온
mean_temp = train.loc[:,"Y00":"Y18"].mean(axis=1)

In [8]:
t = train["Y18"].isna() 
null_index = t[t==True].index
# 평균기온으로 Y18 매움
train.loc[null_index, "Y18"] = train.loc[null_index, "Y00":"Y18"].mean(axis=1)

In [9]:
# Drop
train.drop(columns=['X05','X08','X09','X23','X33','X13','X15','X17','X25','X35','Y03', 'Y04'], axis=1, inplace=True)
test.drop(columns=['X05','X08','X09','X23','X33','X13','X15','X17','X25','X35'], axis=1, inplace=True)


In [ ]:
train.columns

In [10]:
def standardization(df):
    mean = np.mean(df)
    std = np.std(df)
    norm = (df - mean) / (std - 1e-07)
    return norm, mean, std

In [11]:
X_name = train.loc[:,"X00":"X39"].columns
X_name

Index(['X00', 'X01', 'X02', 'X03', 'X04', 'X06', 'X07', 'X10', 'X11', 'X12',
       'X18', 'X20', 'X21', 'X22', 'X24', 'X26', 'X27', 'X28', 'X29', 'X30',
       'X31', 'X32', 'X34', 'X36', 'X37', 'X38', 'X39'],
      dtype='object')

In [12]:
train_X_norm, mean, std = standardization(train[X_name]) 

#테스트용 데이터 표준화
test_X_norm = (test[X_name] - mean) / (std - 1e-07)

In [13]:
train_X_norm.shape, train.shape

((4752, 27), (4752, 69))

In [ ]:
train.columns

In [ ]:
train[["H__0","H__23"]]

In [14]:
train2 = pd.concat([train[["H__0","H__1","H__2","H__3","H__4","H__5","H__6","H__7","H__8","H__9","H__10","H__11","H__12","H__13","H__14","H__15","H__16","H__17","H__18","H__19","H__20","H__21","H__22","H__23"]], train_X_norm], axis=1) 
test2 = pd.concat([test[["H__0","H__1","H__2","H__3","H__4","H__5","H__6","H__7","H__8","H__9","H__10","H__11","H__12","H__13","H__14","H__15","H__16","H__17","H__18","H__19","H__20","H__21","H__22","H__23"]], test_X_norm], axis=1) 

In [15]:
#train2 = pd.concat([train['date'], train_X_norm], axis=1) 
#test2  = pd.concat([test['date'] , test_X_norm] , axis=1)

print(train2.shape, test2.shape)

(4752, 51) (11520, 51)


In [16]:
train2.head()

,H__0,H__1,H__2,H__3,H__4,H__5,H__6,H__7,H__8,H__9,...,X28,X29,X30,X31,X32,X34,X36,X37,X38,X39
0,1,0,0,0,0,0,0,0,0,0,...,-2.314698,0.232479,0.159935,-2.110001,-1.766809,-1.015988,-0.274975,0.577780,0.240728,-0.234601
1,1,0,0,0,0,0,0,0,0,0,...,-2.362038,0.261979,0.215689,-2.091884,-1.842695,-1.015988,-0.274975,0.582902,0.289731,-0.234601
2,1,0,0,0,0,0,0,0,0,0,...,-2.362038,0.232479,0.271442,-2.146235,-1.956524,-1.015988,-0.274975,0.582902,0.311510,-0.234601
3,1,0,0,0,0,0,0,0,0,0,...,-2.362038,0.202979,0.350426,-2.200586,-2.013438,-1.015988,-0.274975,0.593145,0.344178,-0.234601
4,1,0,0,0,0,0,0,0,0,0,...,-2.385708,0.202979,0.401533,-2.254937,-2.013438,-1.015988,-0.274975,0.618753,0.371402,-0.234601


In [17]:
test2.head()

,H__0,H__1,H__2,H__3,H__4,H__5,H__6,H__7,H__8,H__9,...,X28,X29,X30,X31,X32,X34,X36,X37,X38,X39
0,1,0,0,0,0,0,0,0,0,0,...,0.265331,-0.387018,1.196016,-0.425120,-0.154235,-1.015988,-0.274975,0.926050,0.093720,-0.234601
1,1,0,0,0,0,0,0,0,0,0,...,0.241661,-0.416518,1.205308,-0.443237,-0.173206,-1.015988,-0.274975,0.926050,0.126389,-0.234601
2,1,0,0,0,0,0,0,0,0,0,...,0.241661,-0.446017,1.223892,-0.479471,-0.192178,-1.015988,-0.274975,0.977266,0.142723,-0.234601
3,1,0,0,0,0,0,0,0,0,0,...,0.170651,-0.446017,1.270353,-0.479471,-0.230121,-1.015988,-0.274975,1.018239,0.164502,-0.234601
4,1,0,0,0,0,0,0,0,0,0,...,0.170651,-0.475517,1.237830,-0.443237,-0.211149,-1.015988,-0.274975,1.095063,0.191725,-0.234601


In [18]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
train['Y18']

In [19]:
lgb_train = lgb.Dataset(train2, label=train['Y18'])

lgb_param = {
    "objective":"regression",
    "metrics":"mae",
    "learning_rate":0.01
}

print("cv start")
cv_result = lgb.cv(
    lgb_param,
    lgb_train,
    num_boost_round=99999,
    nfold=5,
    early_stopping_rounds=10,
    stratified=False,
    verbose_eval=10 
)

print("train start")
lgb_model = lgb.train(
    lgb_param,
    lgb_train,
    num_boost_round=len(cv_result["l1-mean"])
)

cv start
[10]	cv_agg's l1: 5.57455 + 0.0878939
[20]	cv_agg's l1: 5.05889 + 0.0815726
[30]	cv_agg's l1: 4.59146 + 0.0754426
[40]	cv_agg's l1: 4.16932 + 0.0695657
[50]	cv_agg's l1: 3.7881 + 0.0645189
[60]	cv_agg's l1: 3.44402 + 0.0603034
[70]	cv_agg's l1: 3.13452 + 0.0559298
[80]	cv_agg's l1: 2.85575 + 0.0517863
[90]	cv_agg's l1: 2.60554 + 0.0478675
[100]	cv_agg's l1: 2.38041 + 0.0448815
[110]	cv_agg's l1: 2.17856 + 0.0419204
[120]	cv_agg's l1: 1.99753 + 0.0390049
[130]	cv_agg's l1: 1.83564 + 0.0359401
[140]	cv_agg's l1: 1.69056 + 0.0337149
[150]	cv_agg's l1: 1.56116 + 0.0316893
[160]	cv_agg's l1: 1.44523 + 0.0293332
[170]	cv_agg's l1: 1.34116 + 0.0276363
[180]	cv_agg's l1: 1.2479 + 0.0265324
[190]	cv_agg's l1: 1.16437 + 0.0261038
[200]	cv_agg's l1: 1.09052 + 0.0253524
[210]	cv_agg's l1: 1.02445 + 0.0244879
[220]	cv_agg's l1: 0.965388 + 0.0236231
[230]	cv_agg's l1: 0.91303 + 0.0231317
[240]	cv_agg's l1: 0.86731 + 0.0225047
[250]	cv_agg's l1: 0.826302 + 0.0219311
[260]	cv_agg's l1: 0.7891

[2050]	cv_agg's l1: 0.359223 + 0.0141984
[2060]	cv_agg's l1: 0.359053 + 0.0141937
[2070]	cv_agg's l1: 0.358862 + 0.0141554
[2080]	cv_agg's l1: 0.358663 + 0.0141522
[2090]	cv_agg's l1: 0.35848 + 0.0141561
[2100]	cv_agg's l1: 0.358285 + 0.0141739
[2110]	cv_agg's l1: 0.358121 + 0.0141652
[2120]	cv_agg's l1: 0.357948 + 0.0141757
[2130]	cv_agg's l1: 0.357834 + 0.0142143
[2140]	cv_agg's l1: 0.357691 + 0.0142301
[2150]	cv_agg's l1: 0.357487 + 0.0142358
[2160]	cv_agg's l1: 0.357288 + 0.0142739
[2170]	cv_agg's l1: 0.357133 + 0.0142777
[2180]	cv_agg's l1: 0.356932 + 0.0142959
[2190]	cv_agg's l1: 0.356738 + 0.0143229
[2200]	cv_agg's l1: 0.356555 + 0.0143476
[2210]	cv_agg's l1: 0.356407 + 0.0143451
[2220]	cv_agg's l1: 0.356205 + 0.0143465
[2230]	cv_agg's l1: 0.356041 + 0.0143193
[2240]	cv_agg's l1: 0.355842 + 0.0143051
[2250]	cv_agg's l1: 0.355633 + 0.0142676
[2260]	cv_agg's l1: 0.355465 + 0.0142416
[2270]	cv_agg's l1: 0.355296 + 0.0142046
[2280]	cv_agg's l1: 0.355155 + 0.0141686
[2290]	cv_agg's l

[4060]	cv_agg's l1: 0.338328 + 0.0144282
[4070]	cv_agg's l1: 0.338256 + 0.014431
[4080]	cv_agg's l1: 0.338202 + 0.0144396
[4090]	cv_agg's l1: 0.338145 + 0.0144396
[4100]	cv_agg's l1: 0.338091 + 0.0144427
[4110]	cv_agg's l1: 0.338052 + 0.014433
[4120]	cv_agg's l1: 0.338008 + 0.0144234
[4130]	cv_agg's l1: 0.337953 + 0.0144231
[4140]	cv_agg's l1: 0.337884 + 0.0144239
[4150]	cv_agg's l1: 0.337829 + 0.0144161
[4160]	cv_agg's l1: 0.337766 + 0.0144057
[4170]	cv_agg's l1: 0.337718 + 0.0143967
[4180]	cv_agg's l1: 0.337683 + 0.0143879
[4190]	cv_agg's l1: 0.337633 + 0.0144037
[4200]	cv_agg's l1: 0.337585 + 0.0143923
[4210]	cv_agg's l1: 0.33754 + 0.0144053
[4220]	cv_agg's l1: 0.337493 + 0.0144177
[4230]	cv_agg's l1: 0.337438 + 0.0144247
[4240]	cv_agg's l1: 0.337395 + 0.0144327
[4250]	cv_agg's l1: 0.337346 + 0.0144409
[4260]	cv_agg's l1: 0.337301 + 0.0144555
[4270]	cv_agg's l1: 0.337253 + 0.0144517
[4280]	cv_agg's l1: 0.337206 + 0.014439
[4290]	cv_agg's l1: 0.337159 + 0.0144405
[4300]	cv_agg's l1: 

[6070]	cv_agg's l1: 0.331801 + 0.0140336
[6080]	cv_agg's l1: 0.331781 + 0.0140211
[6090]	cv_agg's l1: 0.331766 + 0.014017
[6100]	cv_agg's l1: 0.331753 + 0.0140108
[6110]	cv_agg's l1: 0.331729 + 0.0140146
[6120]	cv_agg's l1: 0.331716 + 0.0140057
[6130]	cv_agg's l1: 0.331704 + 0.0140044
[6140]	cv_agg's l1: 0.331695 + 0.01401
[6150]	cv_agg's l1: 0.33168 + 0.0140118
[6160]	cv_agg's l1: 0.331664 + 0.0140105
[6170]	cv_agg's l1: 0.331649 + 0.0140099
[6180]	cv_agg's l1: 0.331626 + 0.0140063
[6190]	cv_agg's l1: 0.331606 + 0.0140105
[6200]	cv_agg's l1: 0.33159 + 0.0140096
[6210]	cv_agg's l1: 0.331572 + 0.0140018
[6220]	cv_agg's l1: 0.331559 + 0.0140061
[6230]	cv_agg's l1: 0.331543 + 0.0140115
[6240]	cv_agg's l1: 0.33153 + 0.0140041
[6250]	cv_agg's l1: 0.331519 + 0.014002
[6260]	cv_agg's l1: 0.331504 + 0.0140045
[6270]	cv_agg's l1: 0.331483 + 0.0140046
[6280]	cv_agg's l1: 0.331462 + 0.0139979
[6290]	cv_agg's l1: 0.331454 + 0.0140048
[6300]	cv_agg's l1: 0.331437 + 0.0140006
[6310]	cv_agg's l1: 0.3

In [ ]:
#test2.drop(['date'], axis=1, inplace=True)
test2.head()

In [20]:
#submission["Y18"] = lgb_model.predict(test.loc[:, "X00":"X39"])
submission["Y18"] = lgb_model.predict(test2)
submission.head()

,id,Y18
0,4752,20.215906
1,4753,20.276776
2,4754,20.110886
3,4755,19.738449
4,4756,19.652736


In [21]:
submission.to_csv('data/submission_0406_3.csv', index=False)